# Day 16: Flawed Frequency Transmission

https://adventofcode.com/2019/day/16

## Part 1

In [1]:
with open("./data/input16.txt") as f:
    line = [l.rstrip('\n') for l in f]
message = []
for i in range(len(line[0])):
    message.append(int(line[0][i]))

In [2]:
basepattern = [ 0, 1, 0, -1 ]

def pattern(pos=0): # use C numbering starting from 0
    pattern = []
    for p in basepattern:
        for i in range(pos+1):
            pattern.append(p)
    return pattern

def fullpattern(signal,pos):
    fullpattern = []
    basep = pattern(pos)
    np = len(basep)
    j = 1
    for i in range(len(signal)):
        fullpattern.append(basep[j])
        j+=1
        if (j==np): j=0
    return fullpattern

signal = [ 1,2,3,4,5,6,7,8 ]
fullpattern(signal,0)

[1, 0, -1, 0, 1, 0, -1, 0]

In [3]:
def applyPhase(signal):
    output = []
    for i in range(len(signal)):
        sum = 0
        ipattern = fullpattern(signal,i)
        for j in range(len(signal)):
            sum += signal[j]*ipattern[j]
        output.append(abs(sum)%10)
    return output

print(applyPhase(signal))

[4, 8, 2, 2, 6, 1, 5, 8]


In [4]:
def applyNPhases(signal,nphase,verbose=False):
    signal_ = list(signal)
    iphase = 0
    while(iphase<nphase):
        if verbose:
            print("Applying phase #",iphase+1)
        output = applyPhase(signal_)
        signal_ = output
        iphase += 1
    return output

applyNPhases(signal,4)[0:8]

[0, 1, 0, 2, 9, 4, 9, 8]

In [5]:
def vectorToInt(vector):
    decimal = 1
    total = 0
    for d in reversed(vector):
        total += d * decimal;
        decimal *= 10;
    return total

In [6]:
# 80871224585914546619083218645595 becomes 24176176.
# 19617804207202209144916044189917 becomes 73745418.
# 69317163492948606335995924319873 becomes 52432133.

signal = [8,0,8,7,1,2,2,4,5,8,5,9,1,4,5,4,6,6,1,9,0,8,3,2,1,8,6,4,5,5,9,5]
signal = [1,9,6,1,7,8,0,4,2,0,7,2,0,2,2,0,9,1,4,4,9,1,6,0,4,4,1,8,9,9,1,7]
signal = [6,9,3,1,7,1,6,3,4,9,2,9,4,8,6,0,6,3,3,5,9,9,5,9,2,4,3,1,9,8,7,3]

output = applyNPhases(signal,100)[0:8]
print(vectorToInt(output))

52432133


In [7]:
output = applyNPhases(message,100)[0:8]
print(vectorToInt(output))

28430146


## Part 2

Brute force with solution to Part 1 will not work, given the absurd size of the vector.

Some toughts trying to crack the puzzle...

- Given the presence of the offset and its value, it looks I'm only interested in the last ~10th of digits.
Can I maybe start summing from the end of the vector and exploit string recurrency?

- I notice that the pattern for a given digit have a period N = 4 * digit.

- I also notice that, because of the nature of the pattern, the highest pattern is by definition [0,0,...,0,0,1], the second highest is [0,0,....0,1,1] and so on until I reach digit N/2, where a 0 at end appears (N is the lenght of the message). **THIS IS WHERE THE TRICK IS!**

- Since I'm only interested in the digits in the last ~10th of digits, I can simply compute the last digits. **To do so, given the nature of the patterns, I can start from the end and compute only the last N/2 digits as partial sums, and I will only need the last N/2 digits for the next phase! No pattern is really needed since the offsets is saving me! The first N/2 digits will be gibberish, but since I have the offest to select the answer in the last part of the vector I DON'T CARE!**

- The digits repetition is a red herring!

In [8]:
offset = vectorToInt(message[0:7])
ss = (10000*message)
print(len(ss), offset, len(ss)-offset)

6500000 5970417 529583


In [9]:
import numpy as np

In [ ]:
# 03036732577212944063491565474664 becomes 84462026.
# 02935109699940807407585447034323 becomes 78725270.
# 03081770884921959731165446850517 becomes 53553731.

signal = [0,3,0,3,6,7,3,2,5,7,7,2,1,2,9,4,4,0,6,3,4,9,1,5,6,5,4,7,4,6,6,4]

signal = list(message)

offset = vectorToInt(signal[0:7])
fullsignal = (10000*signal)

nphase = 100
t = np.array(fullsignal)
t = t[::-1] # revert array to compute cumulative sums from the end
for ph in range(nphase):
    sums = np.cumsum(t)%10 # cumulative sums correspoding to results from high patterns, plus module 10 to take last digit
    t = sums.copy()
t = t[::-1] # revert array back
output = t[offset:offset+8]

print(vectorToInt(output))